In [27]:
# !pip install simpleitk
import SimpleITK as sitk
import numpy as np
import pydicom
import scipy.ndimage
import random 
import pandas as pd
import os

from skimage.segmentation import slic
from scipy.ndimage import binary_closing, generate_binary_structure

from mirp.extract_features_and_images import extract_features_and_images
from mirp._data_import.utilities import flatten_list

In [3]:
ct_image_path = "/Users/kamleshranabhat/Desktop/NAAMII/Data_nii/S0819_P200944693_0_CT_5_3_CHEST_ABDOMEN_PELVIS_2.nii"
ct_image = sitk.ReadImage(ct_image_path)

RTSTRUCT_path = "/Users/kamleshranabhat/Desktop/NAAMII/Data_nii/S0819_P200944693_0_RTSTRUCT_LUNG_1.nii"

ROI = sitk.ReadImage(RTSTRUCT_path)


rtstruct_array = sitk.GetArrayFromImage(ROI)

ct_array = sitk.GetArrayFromImage(ct_image)

In [4]:
print("CT Image Shape:", ct_array.shape)
print("RTSTRUCT Shape:", rtstruct_array.shape)

CT Image Shape: (128, 512, 512)
RTSTRUCT Shape: (128, 512, 512)


In [15]:
def rotate_3d_image(image, angle):
    """
    Rotates a 3D image by a given angle around all three axes.

    Parameters:
    image (SimpleITK.Image): The input 3D image.
    angle (float): The rotation angle in degrees.

    Returns:
    SimpleITK.Image: The rotated 3D image.
    """
    transform = sitk.Euler3DTransform()
    angle_rad = np.deg2rad(angle)
    transform.SetRotation(angle_rad, angle_rad, angle_rad)

    rotated_image = sitk.Resample(image, transform, sitk.sitkLinear, 0.0, image.GetPixelID())
    return rotated_image

def apply_rotation_and_save(ct_image_path, rtstruct_path, output_filename, angle=10.0):
    """
    Applies rotation to the CT image and ROI, and saves the output.

    Parameters:
    ct_image_path (str): Path to the CT image file.
    rtstruct_path (str): Path to the RTSTRUCT file.
    output_filename (str): The base filename for the output.
    angle (float): The rotation angle in degrees.
    """
    ct_image = sitk.ReadImage(ct_image_path)
    ROI = sitk.ReadImage(rtstruct_path)

    rotated_ct_image = rotate_3d_image(ct_image, angle)
    rotated_roi = rotate_3d_image(ROI, angle)

    sitk.WriteImage(rotated_ct_image, f'/Users/kamleshranabhat/Desktop/NAAMII/Data_nii/rotation_{output_filename}_ct_{angle}.nii')
    sitk.WriteImage(rotated_roi, f'/Users/kamleshranabhat/Desktop/NAAMII/Data_nii/rotation_{output_filename}_roi_{angle}.nii')


In [14]:
ct_image_path = "/Users/kamleshranabhat/Desktop/NAAMII/Data_nii/S0819_P200944693_0_CT_5_3_CHEST_ABDOMEN_PELVIS_2.nii"
rtstruct_path = "/Users/kamleshranabhat/Desktop/NAAMII/Data_nii/S0819_P200944693_0_RTSTRUCT_LUNG_1.nii"
output_filename = "S0819_P200944693_0"

apply_rotation_and_save(ct_image_path, rtstruct_path, output_filename)


saved


In [16]:
def translate_3d_image(image, translation):
    """
    Translates a 3D image by given translations along each axis.

    Parameters:
    image (SimpleITK.Image): The input 3D image.
    translation (tuple): A tuple (tx, ty, tz) representing the translation along x, y, and z axes.

    Returns:
    SimpleITK.Image: The translated 3D image.
    """
    transform = sitk.TranslationTransform(3, translation)
    translated_image = sitk.Resample(image, transform, sitk.sitkLinear, 0.0, image.GetPixelID())
    return translated_image

def apply_translation_and_save(ct_image_path, rtstruct_path, output_filename, translation=(10, 10, 10)):
    """
    Applies translation to the CT image and ROI, and saves the output.

    Parameters:
    ct_image_path (str): Path to the CT image file.
    rtstruct_path (str): Path to the RTSTRUCT file.
    output_filename (str): The base filename for the output.
    translation (tuple): A tuple (tx, ty, tz) representing the translation along x, y, and z axes.
    """
    ct_image = sitk.ReadImage(ct_image_path)
    ROI = sitk.ReadImage(rtstruct_path)

    translated_ct_image = translate_3d_image(ct_image, translation)
    translated_roi = translate_3d_image(ROI, translation)

    sitk.WriteImage(translated_ct_image, f'/Users/kamleshranabhat/Desktop/NAAMII/Data_nii/translation_{output_filename}_ct_{translation}.nii')
    sitk.WriteImage(translated_roi, f'/Users/kamleshranabhat/Desktop/NAAMII/Data_nii/translation_{output_filename}_roi_{translation}.nii')


In [17]:
ct_image_path = "/Users/kamleshranabhat/Desktop/NAAMII/Data_nii/S0819_P200944693_0_CT_5_3_CHEST_ABDOMEN_PELVIS_2.nii"
rtstruct_path = "/Users/kamleshranabhat/Desktop/NAAMII/Data_nii/S0819_P200944693_0_RTSTRUCT_LUNG_1.nii"
output_filename = "S0819_P200944693_0"

apply_translation_and_save(ct_image_path, rtstruct_path, output_filename)


In [5]:
def volume_adaptation(mask, T):
    V0 = np.sum(mask)
    Va = int(np.floor(V0 * (1 + T)))
    
    struct_elem = scipy.ndimage.generate_binary_structure(3, 1)
    Rp = mask.copy()
    Vp = V0

    while True:
        if T > 0.0:
            Rn = scipy.ndimage.binary_dilation(Rp, structure=struct_elem).astype(Rp.dtype)
        else:
            Rn = scipy.ndimage.binary_erosion(Rp, structure=struct_elem).astype(Rp.dtype)
        
        Vn = np.sum(Rn)

        if Vn == 0 or (T > 0.0 and Vn > Va) or (T < 0.0 and Vn < Va):
            break

        Rp = Rn.copy()
        Vp = Vn

    if Vn != Va:
        N = int(abs(Va - Vp))  # Ensure N is an integer
        Rr = np.logical_xor(Rn, Rp).astype(np.uint8)
        rim_voxels = np.column_stack(np.where(Rr))
        if N > 0 and rim_voxels.shape[0] > 0:
            random_voxels = rim_voxels[np.random.choice(rim_voxels.shape[0], N, replace=False)]

            if T > 0.0:
                Rp[tuple(random_voxels.T)] = 1
            else:
                Rp[tuple(random_voxels.T)] = 0

    return Rp


In [8]:
ct_image_path = "/Users/kamleshranabhat/Desktop/Data_nii/S0819_P200944693_0_CT_5_3_CHEST_ABDOMEN_PELVIS_2.nii"
ct_image = sitk.ReadImage(ct_image_path)

RTSTRUCT_path = "/Users/kamleshranabhat/Desktop/Data_nii/S0819_P200944693_0_RTSTRUCT_LUNG_1.nii"
ROI = sitk.ReadImage(RTSTRUCT_path)

ct_image_array = sitk.GetArrayFromImage(ct_image)
ROI_array = sitk.GetArrayFromImage(ROI)

mask = ROI_array.astype(np.uint8)

T = -0.5

adapted_mask = volume_adaptation(mask, T)

adapted_mask_image = sitk.GetImageFromArray(adapted_mask)
adapted_mask_image.CopyInformation(ROI)

adapted_RTSTRUCT_path = "/Users/kamleshranabhat/Desktop/Data_nii/Adapted_RTSTRUCT_LUNG_neg05.nii"
sitk.WriteImage(adapted_mask_image, adapted_RTSTRUCT_path)

print("Original volume:", np.sum(mask))
print("Adapted volume:", np.sum(adapted_mask))


Original volume: 68441
Adapted volume: 34220


In [49]:
def crop_to_roi(image, mask, crop_margin_mm=25):
    spacing = image.GetSpacing()
    margin_voxels = np.array(crop_margin_mm / np.array(spacing), dtype=int)

    image_array = sitk.GetArrayFromImage(image)
    mask_array = sitk.GetArrayFromImage(mask)

    roi_indices = np.where(mask_array > 0)
    min_idx = np.maximum(np.min(roi_indices, axis=1) - margin_voxels, 0)
    max_idx = np.minimum(np.max(roi_indices, axis=1) + margin_voxels, np.array(mask_array.shape) - 1)

    cropped_image_array = image_array[min_idx[0]:max_idx[0]+1, min_idx[1]:max_idx[1]+1, min_idx[2]:max_idx[2]+1]
    cropped_mask_array = mask_array[min_idx[0]:max_idx[0]+1, min_idx[1]:max_idx[1]+1, min_idx[2]:max_idx[2]+1]

    cropped_image = sitk.GetImageFromArray(cropped_image_array)
    cropped_image.SetSpacing(image.GetSpacing())
    cropped_image.SetOrigin(image.GetOrigin())
    cropped_image.SetDirection(image.GetDirection())

    cropped_mask = sitk.GetImageFromArray(cropped_mask_array)
    cropped_mask.SetSpacing(mask.GetSpacing())
    cropped_mask.SetOrigin(mask.GetOrigin())
    cropped_mask.SetDirection(mask.GetDirection())

    return cropped_image, cropped_mask

In [88]:
def add_random_noise_to_mask(mask_array, noise_level=0.01):
    """
    Add random noise to the mask. 
    `noise_level` controls the amount of noise.
    """
    noise = np.random.rand(*mask_array.shape)
    
    noisy_mask = np.where(noise < noise_level, 1, mask_array).astype(np.uint8)
    
    return noisy_mask

def apply_contour_randomization(cropped_image, cropped_mask, noise_level=0.01, output_dir='./'):
    """
    Apply random noise to the mask and save the result.
    """
    cropped_mask_array = sitk.GetArrayFromImage(cropped_mask)
    
    noisy_mask_array = add_random_noise_to_mask(cropped_mask_array, noise_level)
    
    noisy_mask = sitk.GetImageFromArray(noisy_mask_array)
    noisy_mask.SetSpacing(cropped_mask.GetSpacing())
    noisy_mask.SetOrigin(cropped_mask.GetOrigin())
    noisy_mask.SetDirection(cropped_mask.GetDirection())
    
    rotated_roi = rotate_3d_image(noisy_mask, 3)
    translated_roi = translate_3d_image(rotated_roi, (2,2,2))
    
    return translated_roi


In [97]:
def apply_crop_and_randomize(ct_image_path, rtstruct_path, output_filename, crop_margin_mm=25, randomize_repetitions=1, rotation_angle=0.0):
    ct_image = sitk.ReadImage(ct_image_path)
    rtstruct_image = sitk.ReadImage(rtstruct_path)
    
    cropped_ct_image, cropped_rtstruct_image = crop_to_roi(ct_image, rtstruct_image, crop_margin_mm=crop_margin_mm)
    
    randomized_rtstruct_image = apply_contour_randomization(cropped_ct_image, cropped_rtstruct_image)
    
    sitk.WriteImage(cropped_ct_image, f"/Users/kamleshranabhat/Desktop/NAAMII/Data_nii/cropped_ct_S0819_P200944693_0_CT_5_3_CHEST_ABDOMEN_PELVIS_2.nii")
    sitk.WriteImage(cropped_rtstruct_image, f"/Users/kamleshranabhat/Desktop/NAAMII/Data_nii/cropped_roi_S0819_P200944693_0_RTSTRUCT_LUNG_1.nii")
    sitk.WriteImage(randomized_rtstruct_image, f"/Users/kamleshranabhat/Desktop/NAAMII/Data_nii/randomized_rtstruct_S0819_P200944693_0_RTSTRUCT_LUNG_1.nii")


In [98]:
ct_image_path = "/Users/kamleshranabhat/Desktop/NAAMII/Data_nii/S0819_P200944693_0_CT_5_3_CHEST_ABDOMEN_PELVIS_2.nii"
rtstruct_path = "/Users/kamleshranabhat/Desktop/NAAMII/Data_nii/S0819_P200944693_0_RTSTRUCT_LUNG_1.nii"
output_filename = "S0819_P200944693_0"

apply_crop_and_randomize(ct_image_path, rtstruct_path, output_filename, crop_margin_mm=25, randomize_repetitions=2, rotation_angle=45.0)